In [1]:
from lib.hair import HairMaskPipeline
import cv2
import numpy as np
import webcolors


In [2]:

hair_mask_pipeline = HairMaskPipeline()

input_image_path ='source_data/roberta.png'

hair_mask_path = hair_mask_pipeline.generate_hair_mask(image_path=input_image_path, output_mask_path='output/masks/hair_mask.png')

print('Hair mask generated at', hair_mask_path)
# Read the input image and hair mask using cv2
input_image_cv = cv2.imread(input_image_path, cv2.IMREAD_UNCHANGED)
hair_mask_cv = cv2.imread(hair_mask_path, cv2.IMREAD_GRAYSCALE)

# Add an alpha channel to the input image if it doesn't have one
if input_image_cv.shape[2] == 3:
  input_image_cv = cv2.cvtColor(input_image_cv, cv2.COLOR_BGR2BGRA)

color_name = 'blue'
rgb_value = webcolors.name_to_rgb(color_name)

print('RGB value for', color_name, 'is', rgb_value)
print('RGB value for', color_name, 'is', rgb_value.red, rgb_value.green, rgb_value.blue)

# Create a red layer with the same size as the input image
color_layer_cv = np.zeros_like(input_image_cv)
color_layer_cv[:, :, 0] = rgb_value.blue  # Set the blue channel
color_layer_cv[:, :, 1] = rgb_value.green  # Set the green channel
color_layer_cv[:, :, 2] = rgb_value.red  # Set the red channel

# Apply the red color only to the mask area
color_layer_cv[:, :, 3] = hair_mask_cv

# Composite the red layer onto the input image using the mask with 10% alpha
colored_image_cv = input_image_cv.copy()
alpha = 0.15
colored_image_cv[hair_mask_cv > 0] = cv2.addWeighted(input_image_cv[hair_mask_cv > 0], 1 - alpha, color_layer_cv[hair_mask_cv > 0], alpha, 0)

# Save the result
cv2.imwrite('output/colored_hair.png', colored_image_cv)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Hair mask saved to: output/masks/hair_mask.png
Hair mask generated at output/masks/hair_mask.png
RGB value for blue is IntegerRGB(red=0, green=0, blue=255)
RGB value for blue is 0 0 255


True